In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.util import ngrams
import plotly.express as px
import scipy.stats
import math
import statistics
import string 
import re
from collections import defaultdict
from gensim import corpora, models
from nltk.text import Text

8 data (x) 
ranking negative(1-3) positive(4,5) (x) 
single words + Bi-grame + Tri-gram

### Import all the data 

In [2]:
# data importing 
lowes_6026 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3286026.csv')
lowes_0314 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_3380314.csv')
lowes_0511 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_50280511.csv') 
lowes_6162 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_1003066162.csv')
lowes_6209 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5000026209.csv')
lowes_9855 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\lowes.com_5001899855.csv')

In [3]:
# data importing 
homedepot_4796 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_204074796.csv')
homedepot_7071 = pd.read_csv('C:\\Users\\lxf12\\Desktop\\Folder of folders\\Corona New project\\Lowes_Homedepot_Reviews\\homedepot.com_319247071.csv')

In [4]:
# set display option to max to see all information in review column
pd.set_option('display.max_colwidth', -1)

C:\Users\lxf12\AppData\Local\Temp\ipykernel_20740\779606820.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [5]:
lowes_0314.head(2)

,Market_place,URL,Product_name,SKU,Price,Currency,Subject,Author,Date,Review,Stars,like,dislike
0,lowes.com,https://www.lowes.com/pd/American-Standard-Saver-White-WaterSense-Labeled-Elongated-Chair-Height-2-piece-Toilet-12-in-Rough-In-Size/3380314,American Standard Saver White Elongated Chair Height 2-piece WaterSense Toilet 12-in Rough-In Size (Ada Compliant),Model #710FA.101.020,179.0,USD,Need Quality Control,USAF157,"December 7, 2020",Nice design with poor quality control. Tank leaked during install. Had to replace fill valve and remove excess porcelain cement from fill valve opening to make it smooth and round.,1,0,0
1,lowes.com,https://www.lowes.com/pd/American-Standard-Saver-White-WaterSense-Labeled-Elongated-Chair-Height-2-piece-Toilet-12-in-Rough-In-Size/3380314,American Standard Saver White Elongated Chair Height 2-piece WaterSense Toilet 12-in Rough-In Size (Ada Compliant),Model #710FA.101.020,179.0,USD,Good Flush,EddieD,"April 12, 2013","I have bought other toilets for a rental. Sometimes there isn't much water in the bowl. Without enough water in bowl, you may need to clean the bowl much more often than usual, if you know what I mean. This toilet does hold a good amount of water in bowl. As far as people saying they need to flush 2 or 3 times....all you need to do is hold down the flushing handle for an extra 1-2 seconds and you get more water flushing from the tank through the bowl. (Quick flush for # 1 and Hold down lever longer for a long flush for #2). It does have a cheap seat, but I used the seat I had instead. The Wax Ring is good. It is deep for a good seal and it has the flange on it. Don't forget to get a very long water supply line. If not sure how long, just get the longest one they have. If too long, you can just twist it into a circle. You'll never see it behind the toilet. This is a high toilet (17 1/2 inches to top of seat). My wife is 5' 4"" . She said "" any higher she would need a step stool"" but she likes the height. We have been using this for over a month now with no complaints or concerns.",5,21,1


In [6]:
# drop null value
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]
namelist = ['lowes_6026', 'lowes_0314', 'lowes_0511', 'lowes_6162',
                    'lowes_6209', 'lowes_9855', 
                     'homedepot_4796', 'homedepot_7071']
n = 0 

for i in datalist: 
    # drop rows with Not a Number (NaN) and None values
    name = namelist[n]
    n += 1
    print(name + ' has null rows amount: '+ str(i['Review'].isna().sum()))
    i.dropna(subset=['Review'], inplace=True)
    print('after cleaning, ' + name + ' has null rows amount: ' 
          + str(i['Review'].isna().sum()))
    

lowes_6026 has null rows amount: 30
after cleaning, lowes_6026 has null rows amount: 0
lowes_0314 has null rows amount: 10
after cleaning, lowes_0314 has null rows amount: 0
lowes_0511 has null rows amount: 15
after cleaning, lowes_0511 has null rows amount: 0
lowes_6162 has null rows amount: 297
after cleaning, lowes_6162 has null rows amount: 0
lowes_6209 has null rows amount: 76
after cleaning, lowes_6209 has null rows amount: 0
lowes_9855 has null rows amount: 170
after cleaning, lowes_9855 has null rows amount: 0
homedepot_4796 has null rows amount: 186
after cleaning, homedepot_4796 has null rows amount: 0
homedepot_7071 has null rows amount: 9
after cleaning, homedepot_7071 has null rows amount: 0


In [7]:
lowes_9855.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 114
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Market_place  100 non-null    object 
 1   URL           100 non-null    object 
 2   Product_name  100 non-null    object 
 3   SKU           100 non-null    object 
 4   Price         100 non-null    float64
 5   Currency      100 non-null    object 
 6   Subject       100 non-null    object 
 7   Author        95 non-null     object 
 8   Date          100 non-null    object 
 9   Review        100 non-null    object 
 10  Stars         100 non-null    int64  
 11  like          100 non-null    int64  
 12  dislike       100 non-null    int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 10.9+ KB


In [8]:
datalist[0].isna().sum()

Market_place    0 
URL             0 
Product_name    0 
SKU             0 
Price           0 
Currency        0 
Subject         48
Author          11
Date            0 
Review          0 
Stars           0 
like            0 
dislike         0 
dtype: int64

### Separate Negative and Positive Ranking

In [9]:
# 8 data x 2 categories 
try:
    user_select = int(input("Please select negative(0) or positive ranking(1): "))
except: 
    print("Your input is not correct. Please input again according the tips.")

if user_select == 0:
    # get reviews that have given 1, 2, and 3 stars (negative)
    lowes_6026 = lowes_6026.loc[lowes_6026['Stars'] <= 3]
    lowes_0314 = lowes_0314.loc[lowes_0314['Stars'] <= 3]
    lowes_0511 = lowes_0511.loc[lowes_0511['Stars'] <= 3]
    lowes_6162 = lowes_6162.loc[lowes_6162['Stars'] <= 3]
    lowes_6209 = lowes_6209.loc[lowes_6209['Stars'] <= 3]
    lowes_9855 = lowes_9855.loc[lowes_9855['Stars'] <= 3]
    # get reviews that have given 1, 2, and 3 stars (negative)
    homedepot_4796 = homedepot_4796.loc[homedepot_4796['Stars'] <= 3]
    homedepot_7071 = homedepot_7071.loc[homedepot_7071['Stars'] <= 3]
elif user_select == 1:
    # get reviews that have given 4 and 5 stars (positive)
    lowes_6026 = lowes_6026.loc[lowes_6026['Stars'] > 3]
    lowes_0314 = lowes_0314.loc[lowes_0314['Stars'] > 3]
    lowes_0511 = lowes_0511.loc[lowes_0511['Stars'] > 3]
    lowes_6162 = lowes_6162.loc[lowes_6162['Stars'] > 3]
    lowes_6209 = lowes_6209.loc[lowes_6209['Stars'] > 3]
    lowes_9855 = lowes_9855.loc[lowes_9855['Stars'] > 3]
    # get reviews that have given 4 and 5 stars (positive)
    homedepot_4796 = homedepot_4796.loc[homedepot_4796['Stars'] > 3]
    homedepot_7071 = homedepot_7071.loc[homedepot_7071['Stars'] > 3]
else: 
    print("Your input is not correct. Please input again according the tips.")

Please select negative(0) or positive ranking(1): 0


In [10]:
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

In [11]:
lowes_0314.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 391
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Market_place  46 non-null     object 
 1   URL           46 non-null     object 
 2   Product_name  46 non-null     object 
 3   SKU           46 non-null     object 
 4   Price         46 non-null     float64
 5   Currency      46 non-null     object 
 6   Subject       46 non-null     object 
 7   Author        45 non-null     object 
 8   Date          46 non-null     object 
 9   Review        46 non-null     object 
 10  Stars         46 non-null     int64  
 11  like          46 non-null     int64  
 12  dislike       46 non-null     int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 5.0+ KB


### Lower case for all words & Remove punctuation

In [12]:
# download package punkt for word_tokenize
# nltk.download('punkt')
# download package for stopwords
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

### Store them to text list

In [13]:
# before the cleaning 
text_01 = lowes_6026['Review'].to_string(index=False)
# text_01

In [14]:
# Create a translation table with all punctuation characters mapped to None
translator = str.maketrans('', '', string.punctuation)

text = [0] * 8
n = 0 

for i in datalist:
    # convert pandas series data to string
    text[n] = i['Review'].to_string(index=False)
    # convert all characters to lowercase
    text[n] = text[n].lower()
    # remove punctuations using RegexpTokenizer
    text[n] = text[n].translate(translator)
    # Use regex to remove all numbers
    text[n] = re.sub(r'\d+', '', text[n])
    n += 1

In [15]:
# the result has been single words 
# text[0]

In [16]:
type(text[0])

str

In [17]:
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

### Stop words

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    # customize stop_words by including a custom list of words
    custom_stop_words = ['toilet']
#     custom_stop_words = ['new','toilet', 'factory', 'flush', 'sku', 'photos', 'stay', 
#                                      'lid', 'good', 'great', 'handle', 'top', 'high', 'urine']
    stop_words.update(custom_stop_words)
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lxf12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
for i in range(8):
    text[i] = remove_stopwords(text[i])

In [20]:
datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
                    lowes_6209, lowes_9855, 
                     homedepot_4796, homedepot_7071]

### Text Tokenization

In [21]:
# remove punctuations using RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tokens = [0] * 8
for i in range(8):
    tokens[i] = tokenizer.tokenize(text[i])

In [22]:
# tokens[0]

### Bi-gram and Tri-gram

In [23]:
# N-gram generation
N = int(input('How many words do you want to aggregate? (1-8)'))

trigrams = [0] * 8
for i in range(8):
    trigrams[i] = list(ngrams(tokens[i], N))

How many words do you want to aggregate? (1-8)4


In [24]:
# trigrams[6]

In [25]:
# ## Count the trigrams with the same set of words
# trigram_count = defaultdict(set)

# for word in trigrams[6]:
#     key = frozenset(word)
#     trigram_count[key].add(word)

# # Print the trigram counts
# for key, amount in trigram_count.items():
#     count = len(amount)
#     print(f"{key}: {count}")

In [26]:
# connect the three words together 
words = [0] * 8
for i in range(8):
    words[i] = [' '.join(tr) for tr in trigrams[i]]

### Frequency Analysis

In [27]:
# select the data 
n = int(input('Which one do you want to look at? (1-8)'))

word_frequency = FreqDist(trigrams[n-1])

# Identify co-occurring trigrams
cooccurring_trigrams = []
for trigram in word_frequency:
    if word_frequency[trigram] > 1:
        cooccurring_trigrams.append(trigram)

word_frequency.most_common(10)

# word_frequency.plot(30,cumulative=False)
# plt.show()

Which one do you want to look at? (1-8)1


[(('american', 'standard', 'customer', 'service'), 3),
 (('needs', 'cleaned', 'every', 'day'), 2),
 (('american', 'standard', 'mainstream', 'model'), 2),
 (('one', 'market', 'flow', 'master'), 2),
 (('small', 'amount', 'water', 'enters'), 2),
 (('amount', 'water', 'enters', 'bowl'), 2),
 (('water', 'enters', 'bowl', 'enough'), 2),
 (('like', 'facts', 'uses', 'little'), 1),
 (('facts', 'uses', 'little', 'water'), 1),
 (('uses', 'little', 'water', 'chair'), 1)]

### Topic Modeling
#### Latent Dirichlet Allocation (LDA), which is a popular unsupervised learning algorithm used for topic modeling.

In [28]:
from gensim import corpora, models

In [29]:
# Step 1: select the data 
n = int(input('Which one do you want to look at? (1-8)'))

# Step 2: Create a dictionary of words and convert the text into a bag-of-words format
dictionary = corpora.Dictionary([words[n-1]])
corpus = [dictionary.doc2bow([word]) for word in words[n-1]]

# Step 3: Apply LDA to the corpus to identify topics in the text
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)

# Step 4: Print the topics and the top words associated with each topic
for i, topic in lda_model.show_topics(formatted=True, num_topics=3, num_words=4):
    print("Topic {}: {}".format(i, topic))

Which one do you want to look at? (1-8)1
Topic 0: 0.000*"could return would one" + 0.000*"measure current seat height" + 0.000*"install charged delivery fee" + 0.000*"nd faulty lowes ive"
Topic 1: 0.001*"company get parts lowes" + 0.001*"noticed hair line crack" + 0.001*"hair line crack water" + 0.001*"time theres tissues forget"
Topic 2: 0.001*"garbage purchased two one" + 0.001*"water wont flush big" + 0.000*"american standard customer service" + 0.000*"small amount water enters"


### Text Summarization
#### In summary, while topic modeling aims to uncover the underlying themes and topics in a large corpus of text, text summarization aims to provide a condensed version of a longer text, capturing the essential information and key points.
#### Two different approaches are used for Text Summarization
1. Extractive Summarization: In Extractive Summarization, we identify essential phrases or sentences from the original text and extract only these phrases from the text. These extracted sentences would be the summary.
2. Abstractive Summarization: We work on generating new sentences from the original text in the Abstractive Summarization approach. The abstractive method contrasts the approach described above, and the sentences generated through this approach might not even be present in the original text.

In [30]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [31]:
# select the data 
n = int(input('Which one do you want to look at? (1-8)'))
result_text_1 = datalist[n-1]['Review'].to_string(index=False)

Which one do you want to look at? (1-8)1


In [32]:
stopWords = set(stopwords.words("english"))
words = word_tokenize(result_text_1)
freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

In [33]:
sentences = sent_tokenize(result_text_1)
sentenceValue = dict()
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq

In [34]:
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

In [35]:
average = int(sumValues / len(sentenceValue))

In [36]:
# Storing sentences into our summary.
summary = ''
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (2.0 * average)):
        summary += " " + sentence

In [37]:
print(summary)

In [38]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
from spacy.lang.en.examples import sentences 

In [39]:
def summarize(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary

In [40]:
# datalist = [lowes_6026, lowes_0314, lowes_0511, lowes_6162,
#                     lowes_6209, lowes_9855, 
#                      homedepot_4796, homedepot_7071]

In [41]:
# text_6026 = lowes_6026['Review'].to_string(index=False)

In [42]:
print(summarize(result_text_1, 0.03))
# per is the percentage (0 to 1) of sentences you want to extract

THe absoulte most horrible toilet i have ever used... do not waste your money or time on this one...spend the extra money cuz this toilet not only will not flush a leaf but extremly Uncomfortable seat ive ever used in home or in public... complete trash                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

### Context Analysis
#### Check out the context of words having highest frequency

In [43]:
# select the data 
n = int(input('Which one do you want to look at? (1-8)'))
result_text_1 = text[n-1]

Which one do you want to look at? (1-8)1


In [51]:
# pos and lemmatizer

lemmatizer = WordNetLemmatizer()
lemm = lemmatizer.lemmatize(result_text_1)
pos_tag_1 = pos_tag(word_tokenize(lemm))
# pos_tag_1

### select Adjective or Noun 
result_adj_1 = []
for token, tag in pos_tag_1:
    if tag[0:1].lower() == "j":   ### j represents Adj. n represents noun
        result_adj_1.append(token)
print(f"Total number of ADJ: {len(result_adj_1)}\n")
# print(result_adj_1)

Total number of ADJ: 1409



In [52]:
# see each Adjective frequence
ADJ_f_1 = FreqDist(result_adj_1)
# ADJ_f_1
print(f"The top 10 most common Adjective. word appear in review is: {ADJ_f_1.most_common(10)}")

The top 10 most common Adjective. word appear in review is: [('flush', 70), ('american', 36), ('new', 33), ('good', 32), ('old', 23), ('small', 19), ('great', 18), ('little', 17), ('much', 15), ('bad', 15)]


#### concordance function: to locate the key word and get the context
https://www.nltk.org/howto/concordance.html

In [54]:
# find word in the text
# select the data 
# n = int(input('Which one do you want to look at? (1-8)'))
textList_1 = Text(tokens[n-1])
textList_1.concordance("chair", lines=10, width=300)
# textList_1.concordance(['american', 'standard'], lines=10, width=300)

Displaying 3 of 3 matches:
like facts uses little water chair height dont like fact needs cleaned every day flushes water doesnt swirl like toilets stead swirling bubbles looks pretty darn gross solid waste b
told adjust water level said plumber north years know adjust water level ordered new seal replaced works fine saved money replace old toilets ones chair height however horrible everything sticks sides need frequent cleaning choos one power wash feature must missing mine wasted savings flush doesnt 
ng would good fit small space tight budget good fit trying match bright white fixtures bathroom american standard mainstream white gpf round piece chair height embossed tank made mexico worse amerstandard ever made paid plus installation charge worth dime wont take solids till flushes taken market 


In [47]:
# tokens = nltk.word_tokenize(text[n-1])
# concordance = nltk.ConcordanceIndex(tokens)

# search_term = "tank"
# context_width = 300

# concordance.print_concordance(search_term, lines=10, width=context_width)

In [48]:
# # combine with text summarization

# # concordance_lines = concordance.find_concordance(search_term, width=context_width)
# # concordance_string = '.'.join(line.line for line in concordance_lines)

# con_list = textList_1.concordance_list("tank", width=300)
# concordance_string = ' '.join(line.line for line in con_list)
# # context = str(concordance)
# print(summarize(concordance_string, 0.1))
# # per is the percentage (0 to 1) of sentences you want to extract

In [49]:
# concordance_string